In [1]:
import os
import cv2
import io
import pickle
import requests
import zipfile
import numpy as np
from scipy import ndimage
from skimage.transform import resize
from imageio import imread, imsave
from skimage import img_as_float, color, exposure
from skimage.feature import peak_local_max, hog
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import matplotlib.cm as cm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
DATADIR = "./RUSSIACNNTRAIN"
LABELS = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"

data = []
labels = []
subdirs = []
for subdir in sorted(os.listdir(DATADIR)):
    subdirs.append(subdir)
    
    for file in os.listdir(DATADIR+"/"+subdir):
        image_path = DATADIR+"/"+subdir+"/"+file
        image = imread(image_path,as_gray=True)
        hog_features = hog(image, orientations=3, pixels_per_cell=(5, 5),
                cells_per_block=(1, 1),block_norm='L2-Hys')
        data.append(hog_features)
        labels.append(LABELS.index(subdir))




In [3]:
print('Training the SVM')
#create the SVC
clf = LinearSVC(dual=False,verbose=1)
#train the svm
clf.fit(data, labels)

Training the SVM
[LibLinear]

LinearSVC(dual=False, verbose=1)

In [4]:
pickle.dump( clf, open( "russiansvm.detector", "wb" ) )

In [5]:
clf = pickle.load(open("russiansvm.detector","rb"))

In [6]:
test_data = []
alpha_str = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"

DATADIR = "./XARDBASSTEST"
TEST_LABELS='ПацанынаркотикКАЛВшколевкоробкесдавалНаркотаотстойнейквасаУчитвасШколаХардбасаНамхардбасерамдавноНевтемудан'.lower()
print(TEST_LABELS)
print(len(TEST_LABELS))

пацанынаркотиккалвшколевкоробкесдавалнаркотаотстойнейквасаучитвасшколахардбасанамхардбасерамдавноневтемудан
107


In [7]:
actual = [alpha_str.index(letter) for letter in TEST_LABELS]
print("actual",actual)
predictions = []
sorted_files = []
for file in os.listdir(DATADIR):
    sorted_files.append(int(file[:file.index(".")]))
sorted_files.sort()
print(sorted_files)
count = 0
for file in sorted_files:
    file_name = str(file)+".png"
    class_num = alpha_str.index(TEST_LABELS[count])
    image_path = os.path.join(DATADIR,file_name)
    image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    test_data.append([image,class_num])
    count+=1


actual [16, 0, 23, 0, 14, 28, 14, 0, 17, 11, 15, 19, 9, 11, 11, 0, 12, 2, 25, 11, 15, 12, 5, 2, 11, 15, 17, 15, 1, 11, 5, 18, 4, 0, 2, 0, 12, 14, 0, 17, 11, 15, 19, 0, 15, 19, 18, 19, 15, 10, 14, 5, 10, 11, 2, 0, 18, 0, 20, 24, 9, 19, 2, 0, 18, 25, 11, 15, 12, 0, 22, 0, 17, 4, 1, 0, 18, 0, 14, 0, 13, 22, 0, 17, 4, 1, 0, 18, 5, 17, 0, 13, 4, 0, 2, 14, 15, 14, 5, 2, 19, 5, 13, 20, 4, 0, 14]
[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107]


In [8]:
for f in sorted_files:
    filename = str(f)+'.png'
    test_image_path = os.path.join(DATADIR,filename)
    test_image = imread(test_image_path,as_gray=True)
    hog_features = hog(test_image, orientations=3, pixels_per_cell=(5, 5),
                cells_per_block=(1, 1),block_norm='L2-Hys')
    result_type = clf.predict(hog_features.reshape(1,-1))
    predictions.append(result_type[0])

In [9]:
print(len(actual))
print(len(predictions))

print(actual)
print(predictions)

107
107
[16, 0, 23, 0, 14, 28, 14, 0, 17, 11, 15, 19, 9, 11, 11, 0, 12, 2, 25, 11, 15, 12, 5, 2, 11, 15, 17, 15, 1, 11, 5, 18, 4, 0, 2, 0, 12, 14, 0, 17, 11, 15, 19, 0, 15, 19, 18, 19, 15, 10, 14, 5, 10, 11, 2, 0, 18, 0, 20, 24, 9, 19, 2, 0, 18, 25, 11, 15, 12, 0, 22, 0, 17, 4, 1, 0, 18, 0, 14, 0, 13, 22, 0, 17, 4, 1, 0, 18, 5, 17, 0, 13, 4, 0, 2, 14, 15, 14, 5, 2, 19, 5, 13, 20, 4, 0, 14]
[16, 0, 23, 2, 14, 29, 14, 0, 17, 11, 15, 19, 9, 11, 11, 0, 12, 2, 25, 11, 30, 12, 18, 2, 11, 15, 17, 30, 1, 11, 30, 18, 4, 0, 2, 0, 12, 14, 0, 17, 11, 15, 19, 2, 15, 19, 18, 19, 15, 10, 14, 18, 10, 11, 2, 0, 18, 2, 20, 24, 10, 19, 2, 0, 18, 25, 11, 15, 12, 0, 22, 0, 17, 4, 1, 0, 18, 0, 14, 0, 13, 22, 2, 17, 4, 1, 0, 18, 30, 17, 0, 13, 4, 0, 2, 14, 15, 14, 18, 2, 19, 30, 13, 20, 4, 2, 14]


In [10]:
print("SVM Accuracy:")
print(accuracy_score(actual,predictions))

SVM Accuracy:
0.8598130841121495
